## Preparación de datos

In [1]:
import altair as alt
import pandas as pd


#cargar datos
df = pd.read_csv("/content/benjamin_database_ocupada.csv", sep=";", encoding="latin-1")

# Extraer valor númerico de la categoría Marca
def parse_num(x):
    if pd.isna(x):
        return None
    s=str(x).strip()
    s=s.replace(",", ".")
    cleaned=""
    for c in s:
      if c.isdigit() or c in ".-+":
        cleaned+=c
      else:
        break
    try:
      return float(cleaned)
    except ValueError:
      print("error al convertir valor")
      return None

df["puntaje"] = df["Marca"].apply(parse_num)

# Filtrado de filas válidas desde 2019
df = df.dropna(subset=["puntaje", "Año"]).copy()
df = df[df["Año"] >= 2019].copy()
df["Año"] = df["Año"].astype(int)

#Echar un vistazo
df.head(250)



,Categoría,Prueba,Marca,Año,Competencia,Nombre Atleta,puntaje
0,Adulto Femenino,Lanzamiento de bala,16.43 metros,2019,Panamericanos Lima adulto,Ivana Gallardo Cruchet,16.43
1,Adulto Femenino,Lanzamiento de disco,50.85 metros,2019,Panamericanos Lima adulto,Ivana Gallardo Cruchet,50.85
2,Adulto Femenino,Lanzamiento de disco,53.42 metros,2019,Panamericanos Lima adulto,Karen Gallardo Pinto,53.42
3,Adulto Femenino,Lanzamiento de martillo,63.39 metros,2019,Panamericanos Lima adulto,Mariana García Walker,63.39
4,Adulto Femenino,Lanzamiento de jabalina,53.66 metros,2019,Panamericanos Lima adulto,María Paz Ríos Lama,53.66
...,...,...,...,...,...,...,...
199,Adulto Femenino,Lanzamiento de martillo,49.91 metros,2025,Nacional Adulto,Valentina Claveria,49.91
200,Adulto Femenino,Lanzamiento de martillo,48.79 metros,2025,Nacional Adulto,Catalina Heilenkotter,48.79
201,Adulto Femenino,Lanzamiento de martillo,47.92 metros,2025,Nacional Adulto,Maria Gonzalez,47.92
202,Adulto Femenino,Lanzamiento de martillo,44.38 metros,2025,Nacional Adulto,Leslye Torrejon,44.38


## Código Base

In [2]:
# Control interactivo por tipo de prueba
pruebas_options = sorted(df["Prueba"].dropna().unique().tolist())
prueba_dropdown = alt.binding_select(options=pruebas_options, name="Prueba: ")
prueba_select = alt.selection_point(
    fields=["Prueba"],
    bind=prueba_dropdown,
    value=pruebas_options[0] if pruebas_options else None,
    name="seleccion_prueba"
)

#Selector por clic
click = alt.selection_point(
    fields=["Categoría (grupo)"],
    on="click",
    clear="true"
)
# Normalización categorías a Juvenil/Junior Femenino y Adulto Femenino
def map_categoria(c):
    if isinstance(c, str) and c.strip().lower().startswith("juvenil"):
        return "Juvenil/Junior Femenino"
    return "Adulto Femenino"

df["Categoría (grupo)"] = df["Categoría"].apply(map_categoria)

# Colores de Categoría
fixed_colors = alt.Scale(
    domain=["Juvenil/Junior Femenino", "Adulto Femenino"],
    range=["#4463B2", "#B24C44"]
)

# Base del gráfico con ejes y tooltips
base = alt.Chart(df).transform_filter(prueba_select).encode(
    alt.X("Año:O", title="Año", sort="ascending", axis=alt.Axis(labelAngle=0)),
    alt.Y("puntaje:Q", title="Marca (m)", axis=alt.Axis(format=".2f")),
    tooltip=[
        alt.Tooltip("Nombre Atleta:N", title="Atleta"),
        alt.Tooltip("Prueba:N", title="Prueba"),
        alt.Tooltip("Categoría (grupo):N", title="Categoría"),
        alt.Tooltip("Competencia:N", title="Competencia"),
        alt.Tooltip("Marca:N", title="Marca"),
        alt.Tooltip("Año:O", title="Año"),
    ],
    detail="Nombre Atleta:N"
)
# Destacador de selección por clic
points = base.mark_point(
    shape="circle", filled=True, size=85
).encode(
    color=alt.Color("Categoría (grupo):N", title="Categoría", scale=fixed_colors),
    opacity=alt.condition(click, alt.value(1), alt.value(0.2))
)

# Gráfico final
chart = points.properties(
    width=160,
    height=200,
    padding=10,
    title="Lanzamiento femenino por categoría entre 2019-2025"
).add_params(
    prueba_select,
    click
).configure_axis(
    labelFontSize=12,
    titleFontSize=12
).configure_legend(
    titleFontSize=13,
    labelFontSize=12,
    orient="bottom",
    labelColor='white',
    titleColor='white'
).configure_view(
    strokeWidth=0,
    fill='black'
).configure(background='#E6E6E6')

chart


alt.Chart(...)

## Exportar a html y al computador


In [3]:
chart.save("lanzamiento_femenino_2019_2025.html")
from google.colab import files
files.download("lanzamiento_femenino_2019_2025.html")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>